In [3]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

!wget https://github.com/gitskim/Pasion/raw/master/diving.mat
!wget https://raw.githubusercontent.com/gitskim/Pasion/master/Diving.txt

--2019-09-02 00:23:58--  https://github.com/gitskim/Pasion/raw/master/diving.mat
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gitskim/Pasion/master/diving.mat [following]
--2019-09-02 00:24:00--  https://raw.githubusercontent.com/gitskim/Pasion/master/diving.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9591612 (9.1M) [application/octet-stream]
Saving to: ‘diving.mat.1’

diving.mat.1        100%[===================>]   9.15M  --.-KB/s    in 0.1s    

2019-09-02 00:24:00 (65.8 MB/s) - ‘diving.mat.1’ saved [9591612/9591612]

--2019-09-02 00:24:01--  https://raw.githubusercontent.com/gitski

# READ: http://www.cs.virginia.edu/~vicente/vislang/notebooks/pytorch-lab.html


# READ
https://discuss.pytorch.org/t/example-of-many-to-one-lstm/1728/11

https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb

In [4]:
import subprocess
import numpy as np
import os.path
import pickle
import scipy.io as sio

# TODO:
# ANNOTATION_FILE = '/home/suhyunkim011/Pasion/Diving.txt'
ANNOTATION_FILE = 'Diving.txt'

# MAT_FILE = '/home/suhyunkim011/Pasion/diving.mat'
MAT_FILE = 'diving.mat'


def get_pose_labels2():
    # move the pictures to a certain directory and create labels
    contents = sio.loadmat(MAT_FILE)
    # print(len(contents))
    tracked = contents['boxes_tracked_wholevideo']
    # print(tracked)
    # print(len(tracked))
    # (298387, 107)
    # print(tracked.shape)

    # print(tracked[0].shape)

    # batchsize = 298387; timesteps = 202; shape = 104;
    arr_frame = []
    arr_score = np.array([])
    arr_score_flat = []
    arr_difficulty = np.array([])
    arr_difficulty_flat = []

    with open(ANNOTATION_FILE) as filename:
        max_counter = 202
        group_counter = 0
        one_group_counter = 0
        supposed_to_be_counter = 0
        for line in filename:
            #print(line)
            if '#' in line:
                continue

            if 'A' in line:

                group_counter += 1
                line_arr = line.split()
                # print(line_arr[0])
                # print(line_arr[1])
                start = -1
                end = -1

                for i in range(0, 2):
                    if i == 0:
                        start = line_arr[0]
                        start = int(start)

                    elif i == 1:
                        end = line_arr[1]
                        end = int(end)

                start = start - 1

                pose_group = np.zeros([202, 104])

                for i in range(start, end + 1):
                    supposed_to_be_counter += 1
                    converted = tracked[i][:104]
                    #print(f"one_group_counter: {one_group_counter}, converted.shape: {converted.shape}")
                    
                    pose_group[one_group_counter, :] = converted
                 
                    one_group_counter += 1

                if one_group_counter > max_counter:
                    max_counter = one_group_counter
                    print("-------------*********---------WARNING: Max counter changed: %d-------------*********",
                          max_counter)
                # len(pose_group): 169; one_group_counter: 169; max_one_group_counter: 202

                # print(pose_group)
                arr_frame.append(
                    pose_group)  # figure out why it worked when I put it above the if one_group_counter > max_one_group_counter:
                one_group_counter = 0

            if 'Score' in line:
                line_arr = line.split()
                arr_sub_total_score = np.empty(max_counter)
                arr_sub_total_score.fill(float(line_arr[2]))

                arr_sub_difficulty_score = np.empty(max_counter)
                arr_sub_difficulty_score.fill(float(line_arr[3]))

                arr_score = np.append(arr_score, arr_sub_total_score)
                arr_difficulty = np.append(arr_difficulty, arr_sub_difficulty_score)
                # print(arr_sub_total_score)

                arr_score_flat.append(float(line_arr[2]))
                arr_difficulty_flat.append(float(line_arr[3]))

        # print(f'arr_frame: {len(arr_frame)}, arr_score_flat: {len(arr_score_flat)}, arr_difficulty_concat: {len(arr_difficulty_flat)}')
        # print(f'arr_frame: {len(arr_frame)}')
        # print(arr_frame[71])
        print(np.array(arr_frame).shape)
    return np.array(arr_frame), np.array(arr_score_flat), arr_difficulty_flat


arr_frames, arr_scores, arr_difficulty = get_pose_labels2()
print(arr_frames.shape)

(159, 202, 104)
(159, 202, 104)


NOTE: drop_out probability is only used if the number of layers is greater than 1. 

In [0]:
train_on_gpu=torch.cuda.is_available()

def pearsonr(x, y):
    """
    Mimics `scipy.stats.pearsonr`

    Arguments
    ---------
    x : 1D torch.Tensor
    y : 1D torch.Tensor

    Returns
    -------
    r_val : float
        pearsonr correlation coefficient between x and y
    
    Scipy docs ref:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html
    
    Scipy code ref:
        https://github.com/scipy/scipy/blob/v0.19.0/scipy/stats/stats.py#L2975-L3033
    Example:
        >>> x = np.random.randn(100)
        >>> y = np.random.randn(100)
        >>> sp_corr = scipy.stats.pearsonr(x, y)[0]
        >>> th_corr = pearsonr(torch.from_numpy(x), torch.from_numpy(y))
        >>> np.allclose(sp_corr, th_corr)
    """
    mean_x = torch.mean(x)
    mean_y = torch.mean(y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    r_num = xm.dot(ym)
    r_den = torch.norm(xm, 2) * torch.norm(ym, 2)
    r_val = r_num / r_den
    return r_val

def corrcoef(x):
    """
    Mimics `np.corrcoef`

    Arguments
    ---------
    x : 2D torch.Tensor
    
    Returns
    -------
    c : torch.Tensor
        if x.size() = (5, 100), then return val will be of size (5,5)

    Numpy docs ref:
        https://docs.scipy.org/doc/numpy/reference/generated/numpy.corrcoef.html
    Numpy code ref: 
        https://github.com/numpy/numpy/blob/v1.12.0/numpy/lib/function_base.py#L2933-L3013

    Example:
        >>> x = np.random.randn(5,120)
        # result is a (5,5) matrix of correlations between rows
        >>> np_corr = np.corrcoef(x)
        >>> th_corr = corrcoef(torch.from_numpy(x))
        >>> np.allclose(np_corr, th_corr.numpy())
        # [out]: True
    """
    # calculate covariance matrix of rows
    mean_x = torch.mean(x, 1)
    xm = x.sub(mean_x.expand_as(x))
    c = xm.mm(xm.t())
    c = c / (x.size(1) - 1)

    # normalize covariance matrix
    d = torch.diag(c)
    stddev = torch.pow(d, 0.5)
    c = c.div(stddev.expand_as(c))
    c = c.div(stddev.expand_as(c).t())

    # clamp between -1 and 1
    # probably not necessary but numpy does it
    c = torch.clamp(c, -1.0, 1.0)

    return c
  



In [15]:
sequence_length = 202
batch_size = 159

class PoseLSTM(nn.Module):
  
  def __init__(self, input_size=104, hidden_size=256, num_layers=1, drop_prob=0.5, lr=0.001, output_size=1):
    super().__init__()
    
    self.drop_prob = drop_prob
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.lr = lr
    self.input_size = input_size
    self.lstm = nn.LSTM(input_size=self.input_size, 
                        hidden_size=self.hidden_size, 
                        num_layers=self.num_layers, 
                        dropout=self.drop_prob, 
                        batch_first=True)
    # why is there an extra dropout layer when there is dropout in lstm?
    self.dropout = nn.Dropout(drop_prob)
    self.fc = nn.Linear(hidden_size, output_size)
    
    
  def forward(self, x, hidden):
    lstm_output, hidden = self.lstm(x, hidden)
    # print(f"suhyun-{lstm_output.shape}") [159, 202, 256]
    # stacking up the outputs of the LSTM
    lstm_out = lstm_output.contiguous().view(-1, self.hidden_size)
    nondropped = self.dropout(lstm_out)
    output = self.fc(nondropped)
    # print(f"suhyun-lstm-fc: {output.shape}") [32118, 1]
    # return the last output for each batch
    output = output.view(batch_size, -1)
    # print(f'output: {output.shape}') [159, 202]
    output = output[:, -1]
    # print(f'forward returns: {output.shape}') [159]
    return output, hidden
  
  # An LSTM has a hidden and a cell state that are saved as a tupel hidden.
  # 
  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data
    
    if (train_on_gpu):
      hidden = (weight.new(self.num_layers, batch_size, self.num_hidden).zero_().cuda(),
               weight.new(self.num_layers, batch_size, self_num_hidden).zero_().cuda())
      
    else:
      hidden = (weight.new(self.num_layers, batch_size, self.num_hidden).zero_(),
             weight.new(self.num_layers, batch_size, self_num_hidden).zero_())
      
    return hidden
  
  
  
model = PoseLSTM()
print(model)
pose_estimation = torch.FloatTensor(arr_frames)
labels = torch.FloatTensor(arr_scores)

epochs=3
criterion = pearsonr
optimizer = torch.optim.Adam(model.parameters())

# print(labels.shape)
# print(model_output.shape)
# print(model_output[0])
for epoch in range(epochs):
  
  # batch is not separated, because the size is small enough and I need to full gradient descent. 

  # initialize hidden state
  h = model.init_hidden(159)
  
  # creating new variables for the hidden state, otherwise we'd backprop
  # through the entire training history
  h = tuple([each.data for each in h])
  
  # 0. clear gradients
  model.zero_grad()
  
  # 1. get the output from the model
  model_output, h = model.forward(pose_estimation, h)
  
  # 2. calculate the loss
  loss = criterion(model_output, labels)
  print("hi")
  # 3. call loss.backward()
  loss.backward()
  
  # 4. update the parameters
  optimizer.step()
  

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


PoseLSTM(
  (lstm): LSTM(104, 256, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)
hi
hi
hi


In [6]:
hidden_state = 256
sequence = 202
batch_size = 159
output_size = 1

output = torch.randn([batch_size, sequence, hidden_state])
print(output.shape)
layer = nn.Linear(hidden_state, output_size)
aoutput = layer(output)
print(aoutput.shape)

#ideal_output = [159, 1]
mod = output.contiguous().view(-1, hidden_state)
print(mod.shape)

torch.Size([159, 202, 256])
torch.Size([159, 202, 1])
torch.Size([32118, 256])


In [8]:

layer2 = nn.Linear(hidden_state, output_size)
result = layer2(mod)
print(result.shape)


torch.Size([32118, 1])
